In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
import sys
import tensorflow.keras
import tensorflow as tf
print("Tensorflow version: {tf.__version__}")
print("Keras version: {tensorflow.keras.__version__}")
print()

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is","available" if gpu else "Not Available")


Tensorflow version: {tf.__version__}
Keras version: {tensorflow.keras.__version__}

GPU is available


In [3]:
!pip install tensorflow-metal 

  Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (1.2 kB)
Using cached tensorflow_metal-1.1.0-cp311-cp311-macosx_12_0_arm64.whl (1.4 MB)


In [3]:
batchsize=8


In [4]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory('MRL eye data/prepared_data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory('MRL eye data/prepared_data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 62693 images belonging to 2 classes.
Found 15673 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory('MRL eye data/prepared_data/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 6532 images belonging to 2 classes.


In [6]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

2023-11-21 16:04:31.791488: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-21 16:04:31.791534: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-11-21 16:04:31.791553: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-11-21 16:04:31.791598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-21 16:04:31.791629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalization[0][0

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint('models',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]


In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples kolk//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


/var/folders/k4/f3knywfd5_g7myz2fpm0hllw0000gn/T/ipykernel_48693/3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
2023-11-21 16:05:00.051269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7835/7836 [============================>.] - ETA: 0s - loss: 0.2362 - accuracy: 0.9116
Epoch 1: val_loss improved from inf to 0.27727, saving model to models
INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets


7836/7836 [==============================] - 453s 57ms/step - loss: 0.2362 - accuracy: 0.9116 - val_loss: 0.2773 - val_accuracy: 0.8854 - lr: 0.0010
Epoch 2/5
7835/7836 [============================>.] - ETA: 0s - loss: 0.2182 - accuracy: 0.9189
Epoch 2: val_loss improved from 0.27727 to 0.24813, saving model to models
INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets


7836/7836 [==============================] - 460s 59ms/step - loss: 0.2181 - accuracy: 0.9189 - val_loss: 0.2481 - val_accuracy: 0.8972 - lr: 0.0010
Epoch 3/5
7836/7836 [==============================] - ETA: 0s - loss: 0.2239 - accuracy: 0.9190
Epoch 3: val_loss did not improve from 0.24813
7836/7836 [==============================] - 429s 55ms/step - loss: 0.2239 - accuracy: 0.9190 - val_loss: 0.2757 - val_accuracy: 0.8857 - lr: 0.0010
Epoch 4/5
7836/7836 [==============================] - ETA: 0s - loss: 0.2255 - accuracy: 0.9201
Epoch 4: val_loss did not improve from 0.24813
7836/7836 [==============================] - 434s 55ms/step - loss: 0.2255 - accuracy: 0.9201 - val_loss: 0.4506 - val_accuracy: 0.8540 - lr: 0.0010
Epoch 5/5
7835/7836 [============================>.] - ETA: 0s - loss: 0.2305 - accuracy: 0.9196
Epoch 5: val_loss improved from 0.24813 to 0.21857, saving model to models
INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets


7836/7836 [==============================] - 1002s 128ms/step - loss: 0.2306 - accuracy: 0.9196 - val_loss: 0.2186 - val_accuracy: 0.9111 - lr: 0.0010


In [14]:
! pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 42.1 MB/s eta 0:00:0000:0100:01


In [12]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

/var/folders/k4/f3knywfd5_g7myz2fpm0hllw0000gn/T/ipykernel_48693/713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.18960832059383392
0.9269455671310425


In [13]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

/var/folders/k4/f3knywfd5_g7myz2fpm0hllw0000gn/T/ipykernel_48693/4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.21807774901390076
0.9085050821304321


In [14]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

/var/folders/k4/f3knywfd5_g7myz2fpm0hllw0000gn/T/ipykernel_48693/3655471885.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.18960832059383392
0.9269455671310425
